In [1]:
import numpy as np
import tensorflow as tf
import os
import pickle
import numpy.random as rnd
from sklearn.utils import shuffle

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load in data

In [2]:
import pickle
import boto3
from io import BytesIO

In [3]:
s3 = boto3.resource('s3')
data_subsets = ['train', 'val']
data = {}

for name in data_subsets:
    with BytesIO() as files:
        path = "omniglot_images/" +name+ ".pickle"
        s3.Bucket("research-paper-omniglot-data").download_fileobj(path, files)
        files.seek(0)    # move back to the beginning after writing
        (X,c) = pickle.load(files)
        data[name] = X

path = '../omniglot_images/'
data_subsets = ["train", "val", "test"]

data = {}
categories = {}
info = {}
        
for name in data_subsets:
    file_path = os.path.join(path, name + ".pickle")
    print("loading data from {}".format(file_path))
    with open(file_path,"rb") as f:
        (X,c) = pickle.load(f)
        data[name] = X
        categories[name] = c

In [4]:
def create_train_data(size, s='train'):
    #get train data and shape
    X=data[s]
    n_classes, n_examples, w, h = X.shape
    
    #initialize 2 empty arrays for the input size in a list
    pairs=[np.zeros((size, h, w,1)) for i in range(2)]
    
    #initialize vector for the targets
    targets=np.zeros((size,1))
    
    for x in range(size):
        #randomly sample one class (character)
        category = rnd.choice(n_classes,1,replace=False)
        #randomly sample one example from class (1-20 characters)
        idx_1 = rnd.randint(0, n_examples)
        pairs[0][x,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        #randomly sample again one example from class and add last class with modulo
        # ..to ensure not same class pairs are created
        idx_2 = (idx_1 + rnd.randint(0, n_examples)) % n_examples
        #pick images of different class for 1st half and same class for 2nd half
        if x >= size // 2:
            category_2 = category
            targets[x] = 1
        else: 
        #add a random number to the category modulo n classes to ensure 2nd image has
        # ..different category
            idx_2 = rnd.randint(0, n_examples) 
            category_2 = (category + rnd.randint(1,n_classes)) % n_classes
            targets[x] = 0
        pairs[1][x,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
        
    return pairs, targets

In [5]:
train_set, train_labels = create_train_data(10000)
val_set, val_labels = create_train_data(3000)

## Settings

In [6]:
caps1_size = 8
caps2_size = 32 # actually number of capsules
pred_matrix_size = 16 # this is actually capsule size of 2
conv1_channels = 256
conv1_filter = 34
primaryCaps_channels = 32
primaryCaps_filter = 14
routing_rounds = 3
epsilon = 1e-7
learning_rate = 0.001
fully_layer_size = 48

## Squash function

In [7]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

## Convolutional layer

In [8]:
def convolutional(input_data, conv_shape, stride_shape, name, relu=True):
    weights = tf.get_variable('W'+name, initializer=tf.truncated_normal(conv_shape, stddev=0.3))
    bias = tf.get_variable('B'+name, initializer=tf.truncated_normal([conv_shape[-1]], stddev=0.3))
    out_layer = tf.nn.conv2d(input_data, weights, stride_shape, padding = 'VALID')
    out_layer_bias = tf.add(out_layer, bias)
    
    if relu == True:
        out_layer_final = tf.nn.relu(out_layer_bias)
        return(out_layer_final)
    
    return(out_layer_bias)

## Primarycaps

In [9]:
def primarycaps(input_data, conv_shape, stride_shape, primaryCaps_channels, caps1_size, caps2_size, pose_size, batch):
    output = convolutional(input_data, conv_shape, stride_shape, relu=False, name='primaryCaps')
    filter_size = output.get_shape().as_list()[1]
    caps1_raw = tf.reshape(output, [-1,filter_size*filter_size*primaryCaps_channels,caps1_size], name='caps1_raw')
    caps1_output = squash(caps1_raw, axis=-1)
    caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
    caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
    caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,caps2_size,1,1], name = 'caps1_out_tiled')
    
    weight_matrix = tf.get_variable('Weight_matrix', initializer=tf.truncated_normal([filter_size*filter_size*primaryCaps_channels, caps2_size, pose_size, caps1_size], stddev=0.1))
    #weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')
    #caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')
    caps2_predicted = tf.einsum('abdc,iabcf->iabdf', weight_matrix, caps1_output_expand2_tiled)
    
    return(caps2_predicted)

## Routing by agreement

In [10]:
def routing_by_agreement(input_data, caps2_size, rounds, batch):
    raw_weights = tf.zeros([batch, input_data.get_shape().as_list()[1], caps2_size, 1, 1], name = 'raw_weights')
    
    for i in range(rounds):
        routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
        weighted_predictions = tf.multiply(routing_weights, input_data, name = 'weighted_predictions' + str(i))
        weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
        caps2_output = squash(weighted_sum, axis=-2)
    
        #caps2_output_tiled = tf.tile(caps2_output, [1, input_data.get_shape().as_list()[1], 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
        #agreement = tf.matmul(input_data, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
        agreement = tf.einsum('iabcd,ifbcd->iabcd', input_data, caps2_output)
        raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))
        
    return(caps2_output)

In [11]:
def create_dense_layer(input_data, input_shape, neurons, name):
    weights = tf.get_variable(name+'_W', initializer=tf.truncated_normal([input_shape, neurons], stddev=0.2))
    bias = tf.get_variable(name+'b', initializer=tf.truncated_normal([neurons], mean=0.5, stddev=0.02))
    fully_connected = tf.add(tf.matmul(input_data, weights), bias)
    out_dense_activation = tf.nn.sigmoid(fully_connected)
    
    return(out_dense_activation)

## Graph

In [12]:
def create_network(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,X,fully_layer_size):
    conv1 = convolutional(X, [conv1_filter,conv1_filter,X.get_shape().as_list()[-1],
                                  conv1_channels],[1,3,3,1], name='conv')
    primary = primarycaps(conv1, [primaryCaps_filter,primaryCaps_filter,conv1.get_shape().as_list()[-1],
                              primaryCaps_channels*caps1_size], [1,2,2,1],primaryCaps_channels,
                      caps1_size,caps2_size, pred_matrix_size, batch=tf.shape(X)[0])
    output = routing_by_agreement(primary, caps2_size, routing_rounds, batch=tf.shape(X)[0])
    flat = tf.reshape(output, [-1, caps2_size*pred_matrix_size])
    fully_connected = create_dense_layer(flat, caps2_size*pred_matrix_size , fully_layer_size, 'fully')
    
    return(fully_connected)

In [13]:
tf.reset_default_graph()

In [14]:
X1 = tf.placeholder(tf.float32, [None, 105, 105, 1])
X2 = tf.placeholder(tf.float32, [None, 105, 105, 1])
y = tf.placeholder(tf.float32, [None, 1])

In [15]:
with tf.variable_scope('siamese') as scope:
    output1 = create_network(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,X1,fully_layer_size)
    scope.reuse_variables()
    output2 = create_network(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,X2,fully_layer_size)


l1_dis = tf.abs(tf.subtract(output1, output2))

weights = tf.Variable(tf.truncated_normal([fully_layer_size, 1], stddev=0.03), name='w_final')
bias = tf.Variable(tf.truncated_normal([1], stddev=0.01), name='b_final')
fully_final = tf.add(tf.matmul(l1_dis, weights), bias)
y_estimate = tf.nn.sigmoid(fully_final)    

In [16]:
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = fully_final))

In [17]:
optimiser = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

## Prediction

In [18]:
correct_prediction = tf.equal(tf.round(y_estimate), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

## Setup training

In [19]:
init_op = tf.global_variables_initializer()
epochs = 10
batch_size = 50

#If run on AWS
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True


with tf.Session(config=config) as sess:
    init_op.run()

    total_batch = int(10000/batch_size)
    total_batch_val = int(3000/batch_size)

    for epoch in range(epochs):
        avg_cost = 0
        acc = 0
        for i in range(total_batch):
            batch_x1, batch_x2, batch_y = shuffle(train_set[0],train_set[1], train_labels, n_samples = batch_size)
            a, c, accur= sess.run([optimiser, cross_entropy, accuracy], feed_dict={X1: batch_x1, X2: batch_x2, y: batch_y})
            avg_cost += c/total_batch
            acc += accur
            print('batch:', i)
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        print('avg_cost:', avg_cost)
        print('train_acc:', acc/total_batch)
        acc_val = 0
        for iterations in range(total_batch_val):
            batch_x1, batch_x2, batch_y = shuffle(val_set[0],val_set[1], val_labels, n_samples = batch_size)
            val_acc = sess.run(accuracy, feed_dict={X1: batch_x1, X2: batch_x2, y: batch_y})
            acc_val += val_acc
        print('val_acc:', acc_val/total_batch_val)

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100

batch: 155
batch: 156
batch: 157
batch: 158
batch: 159
batch: 160
batch: 161
batch: 162
batch: 163
batch: 164
batch: 165
batch: 166
batch: 167
batch: 168
batch: 169
batch: 170
batch: 171
batch: 172
batch: 173
batch: 174
batch: 175
batch: 176
batch: 177
batch: 178
batch: 179
batch: 180
batch: 181
batch: 182
batch: 183
batch: 184
batch: 185
batch: 186
batch: 187
batch: 188
batch: 189
batch: 190
batch: 191
batch: 192
batch: 193
batch: 194
batch: 195
batch: 196
batch: 197
batch: 198
batch: 199
Epoch: 4 cost = 0.332
avg_cost: 0.3323831871151926
train_acc: 0.8842999961972237
val_acc: 0.8039999941984812
batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch:

batch: 100
batch: 101
batch: 102
batch: 103
batch: 104
batch: 105
batch: 106
batch: 107
batch: 108
batch: 109
batch: 110
batch: 111
batch: 112
batch: 113
batch: 114
batch: 115
batch: 116
batch: 117
batch: 118
batch: 119
batch: 120
batch: 121
batch: 122
batch: 123
batch: 124
batch: 125
batch: 126
batch: 127
batch: 128
batch: 129
batch: 130
batch: 131
batch: 132
batch: 133
batch: 134
batch: 135
batch: 136
batch: 137
batch: 138
batch: 139
batch: 140
batch: 141
batch: 142
batch: 143
batch: 144
batch: 145
batch: 146
batch: 147
batch: 148
batch: 149
batch: 150
batch: 151
batch: 152
batch: 153
batch: 154
batch: 155
batch: 156
batch: 157
batch: 158
batch: 159
batch: 160
batch: 161
batch: 162
batch: 163
batch: 164
batch: 165
batch: 166
batch: 167
batch: 168
batch: 169
batch: 170
batch: 171
batch: 172
batch: 173
batch: 174
batch: 175
batch: 176
batch: 177
batch: 178
batch: 179
batch: 180
batch: 181
batch: 182
batch: 183
batch: 184
batch: 185
batch: 186
batch: 187
batch: 188
batch: 189
batch: 190